# Pràctica 3 - PLH

### Realitzada pels alumnes Lluc Furriols i Pau Prat Moreno

In [19]:
'''
import os 
f = open("/dev/null", "w")
os.dup2(f.fileno(), 2)
f.close()

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
'''

'\nimport os \nf = open("/dev/null", "w")\nos.dup2(f.fileno(), 2)\nf.close()\n\nimport nltk\nimport ssl\n\ntry:\n    _create_unverified_https_context = ssl._create_unverified_context\nexcept AttributeError:\n    pass\nelse:\n    ssl._create_default_https_context = _create_unverified_https_context\n\nnltk.download()\n'

## Importació de llibreries

In [1]:
import nltk
nltk.download('punkt', quiet=True) # Tokenitzador
nltk.download('averaged_perceptron_tagger', quiet=True) # Etiquetador POS
nltk.download('maxent_ne_chunker', quiet=True) # Etiquetador Entitats Anomenades
nltk.download('words', quiet=True)


True

## Carreguem les dades

In [2]:
nltk.download('conll2002')
from nltk.corpus import conll2002

train_esp = conll2002.iob_sents('esp.train') # Train, 
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\llucfurriols\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [3]:
train_esp

[[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')], [('-', 'Fg', 'O')], ...]

In [3]:
def get_token(sequence):
    """
    Retorna una llista de tokens.
    """
    return [[(token) for token, pos, entity in sentence] for sentence in sequence]

def get_token_POS(sequence):
    """
    Retorna una llista de tokens i el seu POS tag.
    """
    return [[(token, pos) for token, pos, entity in sentence] for sentence in sequence]

def get_token_entity(sequence):
    """
    Retorna una llista de tokens i les seves entitats.
    """
    return [[(token, entity) for token, pos, entity in sentence] for sentence in sequence]

# First execution with no modifications

In [5]:
from nltk.tag import CRFTagger
import pycrfsuite

ct_basic = CRFTagger(feature_func=None)

# Train and test sets without the postag
train_esp_first = get_token_entity(train_esp)
test_esp_first = get_token_entity(test_esp)
print("Quina forma tenen les nostres dades d'entrenament: ",train_esp_first[0])

ct_basic.train(train_esp_first, 'model.crf.tagger')

Quina forma tenen les nostres dades d'entrenament:  [('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]


###  Probar el model en el conjunt de test

In [6]:
# Predir les entitats del conjunt de test
y_pred = ct_basic.tag_sents(get_token(test_esp))
print(y_pred[:2])

[[('La', 'B-LOC'), ('Coruña', 'I-LOC'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B-ORG'), (')', 'O'), ('.', 'O')], [('-', 'O')]]


In [7]:
# Les entitats reals del conjunt de test
y_real = get_token_entity(test_esp)
print(y_real[:2])

[[('La', 'B-LOC'), ('Coruña', 'I-LOC'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B-ORG'), (')', 'O'), ('.', 'O')], [('-', 'O')]]


In [4]:
def extract_entities(tagged_words, encoding='BIO'):
    """
    Extreu les entitats d'una llista de paraules etiquetades segons l'encoding especificat.
    
    Arguments:
        tagged_words: una llista de tuples (word, tag), on 'word' és una paraula del text i 'tag' és la seva etiqueta (BIO/BIOE/BIOW/IO).
        encoding: el tipus de codificació utilitzat per les etiquetes ('BIO', 'BIOW', 'BIOE', 'IO').
        
    Retorna:
        Una llista de tuples (start_index, end_index, entity_type) que representen les entitats trobades.
        'start_index' i 'end_index' són els índexs on comença i acaba l'entitat en la llista de paraules, i 'entity_type' és el tipus d'entitat.
    """

    entities = []  # Llista on guardarem les entitats trobades
    current_entity = []  # Guarda les paraules de l'entitat actual
    current_type = None  # Tipus de l'entitat actual
    current_start_index = None  # Índex d'inici de l'entitat actual

    for index, (word, tag) in enumerate(tagged_words):
        tag_type = None if tag == 'O' else tag[2:]

        if tag == 'O':
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
                current_entity = []
                current_type = None
            continue

        if encoding == 'IO':
            if tag_type != current_type:
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                current_entity = [word]
                current_start_index = index
                current_type = tag_type
            else:
                current_entity.append(word)
        else:
            tag_prefix = tag[:1]
            if tag_prefix in ['B', 'W']:  # Començament d'una nova entitat o entitat de paraula única
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                current_entity = [word]
                current_start_index = index
                current_type = tag_type
                if tag_prefix == 'W':  # Si és una entitat de paraula única, la tanquem immediatament
                    entities.append((current_start_index, index, current_type))
                    current_entity = []
                    current_type = None
            elif tag_prefix == 'I' and current_type == tag_type:
                current_entity.append(word)
            elif encoding == 'BIOE' and tag_prefix == 'E' and current_type == tag_type:
                current_entity.append(word)
                entities.append((current_start_index, index, current_type))
                current_entity = []
                current_type = None
            else:
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                    current_entity = []
                current_type = None

    if current_entity:
        entities.append((current_start_index, index, current_type))

    return entities

# Exemple d'ús
tagged_words = get_token_entity(train_esp)[3]

entities = extract_entities(tagged_words, "BIO")
print(entities)

[(3, 4, 'PER'), (13, 14, 'ORG'), (18, 18, 'LOC'), (20, 20, 'LOC'), (59, 59, 'MISC')]


In [21]:
def evaluate_entities(true_entities, pred_entities):
    """
    Avaluació de les entitats reconegudes comparant conjunts d'entitats.

    Args:
    true_entities (list): Llista de tuples representant les entitats reals (start, end, type).
    pred_entities (list): Llista de tuples representant les entitats predites (start, end, type).

    Returns:
    dict: Un diccionari amb les mètriques 'precision', 'recall', i 'f1_score'.
    """
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true_sent, pred_sent in zip(true_entities, pred_entities):
        true_set = set(true_sent)
        pred_set = set(pred_sent)

        true_positives += len(true_set & pred_set)
        false_positives += len(pred_set - true_set)
        false_negatives += len(true_set - pred_set)

    precisio = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * precisio * recall / (precisio + recall) if (precisio + recall) > 0 else 0


    return {
        'precision': precisio,
        'recall': recall,
        'f1_score': f1_score
    }
    

In [40]:


y_real_entities = [extract_entities(sent) for sent in y_real]
y_pred_entities = [extract_entities(sent) for sent in y_pred]

metrics = evaluate_entities(y_real_entities, y_pred_entities)
print(f'Precision: {metrics["precision"]:.6f} %')
print(f'Recall: {metrics["recall"]:.6f} %')
print(f'F1 Score: {metrics["f1_score"]:.6f} %')



Precision: 0.744405 %
Recall: 0.710512 %
F1 Score: 0.727064 %


# NOSTRES MODIFICACIONS

### Creem un model per predir els postags:

In [19]:
from crf import *
import re
import string

class FeatureGetterPOS:
    def __init__(self):
        self._pattern = re.compile(r"\d")

    def has_digit(self, token):
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        return token[-n:] if len(token) > n else token
    

    def get_features(self, tokens, index):
        
        token = tokens[index]
        features = ["WORD_" + token]

        if token[0].isupper():
            features.append("CAPITALIZATION")

        if self.has_digit(token):
            features.append("HAS_NUM")

        if self.has_punctuation(token):
            features.append("PUNCTUATION")
            
        features.extend(["SUF_" + self.get_suffix(token, n) for n in range(1, 4) if len(token) >= n])
        features.extend(["PRE_" + self.get_prefix(token, n) for n in range(1, 4) if len(token) >= n])
        if index > 0:
            prev_token = tokens[index - 1]
            features.append("PREV_WORD_" + prev_token)
        if index < len(tokens) - 1:
            next_token = tokens[index + 1]
            features.append("NEXT_WORD_" + next_token)
        return features

In [10]:

train_esp_pos = get_token_POS(train_esp)
feature_getter_pos = FeatureGetterPOS()

ct_POS = CRFTagger(feature_func=feature_getter_pos.get_features)
ct_POS.train(train_esp_pos, 'model.crf.taggerPOS')



In [14]:

# Funció per avaluar el POS tagging
def evaluate_POS(true_tags, pred_tags):
    """
    Avaluació del POS tagging comparant seqüències de tags.

    Args:
    true_tags (list): Llista de llistes amb els tags reals.
    pred_tags (list): Llista de llistes amb els tags predits.

    Returns:
    dict: Un diccionari amb les mètriques 'accuracy'.
    """
    total = 0
    correct = 0

    for true_sent, pred_sent in zip(true_tags, pred_tags):
        total += len(true_sent)
        correct += sum(1 for true_tag, pred_tag in zip(true_sent, pred_sent) if true_tag == pred_tag)

    accuracy = correct / total if total > 0 else 0

    return accuracy
  

# Prediccions del POS tagging
y_pred_POS = ct_POS.tag_sents(get_token(test_esp))
y_real_POS = get_token_POS(test_esp)
print(evaluate_POS(y_real_POS, y_pred_POS))


0.9614033725962005


El model predeix el postags amb una accuracy de 96%. Com que és un bon resultat, l'utilitzarem en les feature functions

## Model per predir entitats:

### Codificació BIO

Aquesta classe és igual que la anterior, però utilitza el model anterior per predir els postags

In [15]:

import re
import string
class FeatureGetter2:
    def __init__(self, ct_POS):
        self._pattern = re.compile(r"\d")
        self.ct_POS = ct_POS

    def has_digit(self, token):
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        return token[-n:] if len(token) > n else token
    
    def pos_tag(self, token):
        return self.ct_POS.tag([token])[0][1]


    def get_features(self, tokens, index):
        token = tokens[index]
        features = ["WORD_" + token]
        if token[0].isupper():
            features.append("CAPITALIZATION")
        if self.has_digit(token):
            features.append("HAS_NUM")
        if self.has_punctuation(token):
            features.append("PUNCTUATION")
        features.extend(["SUF_" + self.get_suffix(token, n) for n in range(1, 4) if len(token) >= n])
        features.extend(["PRE_" + self.get_prefix(token, n) for n in range(1, 4) if len(token) >= n])

        pos_tag = self.pos_tag(token)
        features.append("POS_" + pos_tag)
        if index > 0:
            prev_token = tokens[index - 1]
            features.append("PREV_WORD_" + prev_token)
        if index < len(tokens) - 1:
            next_token = tokens[index + 1]
            features.append("NEXT_WORD_" + next_token)
        return features

Probem quins resultats dona sense fer cap modificació al model.

Ara veiem quins resultats dona el model amb les nostres modificacions

## Codificació BIO

In [23]:
from crf import *
train_data = get_token_entity(train_esp)  

# Instància del CRFTagger amb la funció de característiques personalitzada
feature_getter = FeatureGetter2(ct_POS=ct_POS)
ct = CRFTagger(feature_func=feature_getter.get_features)

# Entrenament del model
ct.train(train_data, 'model.crf.tagger')

y_real = get_token_entity(test_esp)
y_pred = ct.tag_sents(get_token(test_esp))

y_real_entities = [extract_entities(sent) for sent in y_real]
y_pred_entities = [extract_entities(sent) for sent in y_pred]

metrics = evaluate_entities(y_real_entities, y_pred_entities)
print(f'Precision: {metrics["precision"]:.6f} %')
print(f'Recall: {metrics["recall"]:.6f} %')
print(f'F1 Score: {metrics["f1_score"]:.6f} %')

Precision: 0.785878 %
Recall: 0.769533 %
F1 Score: 0.777620 %


## Codificació IO

In [32]:
def bio_to_io(bio_tagged_sentences):
    """
    Converteix les etiquetes de múltiples frases des de la codificació BIO a IO.
    
    Arguments:
        bio_tagged_sentences: Una llista de llistes de tuples (word, tag) on 'tag' és en codificació BIO.
    
    Retorna:
        Una llista de llistes de tuples (word, io_tag) on 'io_tag' és en codificació IO per cada frase.
    """
    io_tagged_sentences = []
    
    for sentence in bio_tagged_sentences:
        io_tagged_sentence = []
        for word, tag in sentence:
            if tag.startswith('B-'):
                # Canvia B- per I-
                io_tagged_sentence.append((word, 'I-' + tag[2:]))
            elif tag.startswith('I-'):
                io_tagged_sentence.append((word, tag))
            else:
                # Manté les etiquetes 'O' tal com estan
                io_tagged_sentence.append((word, 'O'))
        io_tagged_sentences.append(io_tagged_sentence)
    
    return io_tagged_sentences


tagged_words = get_token_entity(train_esp)
tagged_words_io = bio_to_io(tagged_words)

print(f"BIO:{tagged_words[3]}")
print(f"IO:{tagged_words_io[3]}")

entities = extract_entities(tagged_words_io[3], "IO")
print(f"Entities IO: {entities}")

BIO:[('La', 'O'), ('petición', 'O'), ('del', 'O'), ('Abogado', 'B-PER'), ('General', 'I-PER'), ('tiene', 'O'), ('lugar', 'O'), ('después', 'O'), ('de', 'O'), ('que', 'O'), ('un', 'O'), ('juez', 'O'), ('del', 'O'), ('Tribunal', 'B-ORG'), ('Supremo', 'I-ORG'), ('del', 'O'), ('estado', 'O'), ('de', 'O'), ('Victoria', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), ('se', 'O'), ('viera', 'O'), ('forzado', 'O'), ('a', 'O'), ('disolver', 'O'), ('un', 'O'), ('jurado', 'O'), ('popular', 'O'), ('y', 'O'), ('suspender', 'O'), ('el', 'O'), ('proceso', 'O'), ('ante', 'O'), ('el', 'O'), ('argumento', 'O'), ('de', 'O'), ('la', 'O'), ('defensa', 'O'), ('de', 'O'), ('que', 'O'), ('las', 'O'), ('personas', 'O'), ('que', 'O'), ('lo', 'O'), ('componían', 'O'), ('podían', 'O'), ('haber', 'O'), ('obtenido', 'O'), ('información', 'O'), ('sobre', 'O'), ('el', 'O'), ('acusado', 'O'), ('a', 'O'), ('través', 'O'), ('de', 'O'), ('la', 'O'), ('página', 'O'), ('CrimeNet', 'B-MISC'), ('.', 'O')]
IO:[('La'